In [4]:
%matplotlib ipympl
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import gsw
from glob import glob

In [5]:
temp_nc = 'https://data.nodc.noaa.gov/thredds/dodsC/nodc/archive/data/0114815/public/temperature/netcdf/decav/1.00/woa13_decav_t00_01.nc'
sal_nc = 'https://data.nodc.noaa.gov/thredds/dodsC/nodc/archive/data/0114815/public/salinity/netcdf/decav/1.00/woa13_decav_s00_01.nc'

In [6]:
temp = xr.open_dataset(temp_nc,decode_times=False)
sal = xr.open_dataset(sal_nc,decode_times=False)

OSError: [Errno -70] NetCDF: DAP server error: b'https://data.nodc.noaa.gov/thredds/dodsC/nodc/archive/data/0114815/public/temperature/netcdf/decav/1.00/woa13_decav_t00_01.nc'

In [5]:
# position of ADCP sent by Crissy
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

In [6]:
f = gsw.f(mlat)

In [7]:
ii = np.array(np.argmin(np.abs(mlat-temp['lat'])))
jj = np.array(np.argmin(np.abs(mlon-temp['lon'])))

NameError: name 'temp' is not defined

In [ ]:
mtemp_an = np.squeeze(temp['t_an'][0,:,ii,jj])
msal_an = np.squeeze(sal['s_an'][0,:,ii,jj])
mtemp_mn = np.squeeze(temp['t_mn'][0,:,ii,jj])
msal_mn = np.squeeze(sal['s_mn'][0,:,ii,jj])
mz = np.array(temp['depth'])

In [ ]:
mpr = gsw.p_from_z(-mz,temp['lat'][ii])
mSA_mn = msal_mn
#mSA_mn = gsw.SA_from_SP(msal_mn, mpr, temp['lon'][jj], temp['lat'][ii])
mCT_mn = gsw.CT_from_t(mSA_mn,mtemp_mn,mpr)
msigma0 = gsw.sigma0(mSA_mn,mCT_mn)
msigma4 = gsw.sigma4(mSA_mn,mCT_mn)

mNsq,mpr_mid = gsw.Nsquared(mSA_mn,mCT_mn,mpr,lat=temp['lat'][ii])
mN = np.sqrt(mNsq)

In [ ]:
ctd_dir = 'data/CTD_data_Station_M/'

file_list = glob(ctd_dir+'*CtdDer.txt')

#filename = '2006347tibrCtdDer.txt'

In [ ]:
mlat = 35+8.4585/60  
mlon = -122-59.9036/60

In [ ]:
dz = 1
salspike_thresh = 0.05

In [4]:
dflist = []

plt.figure()
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

cnt = 0
for i,filename in enumerate(file_list):
    try:
        df = pd.read_csv(filename)
        
        
        #spi, = np.where((np.abs(np.diff(df['salin']))>0.2) | (np.abs(np.diff(df['pres']))>10))
        spi, = np.where((np.abs(np.diff(df['salin'][:-1]))>salspike_thresh) & 
                        (np.abs(np.diff(df['salin'][1:]))>salspike_thresh))
        spi = spi+1
        
        df['salin'][spi] = np.nan
        df['sigmat'][spi] = np.nan
        df['sigmatheta'][spi] = np.nan
        
        df['SA'] = gsw.SA_from_SP(df['salin'], df['pres'], mlon, mlat)
        df['CT'] = gsw.CT_from_t(df['SA'],df['temp'],df['pres'])
        df['sigma0'] = gsw.sigma0(df['SA'],df['CT'])
        df['sigma4'] = gsw.sigma4(df['SA'],df['CT'])
        
        df['zp'] = np.floor(df['pres']/dz+0.5)*dz
        dfm = df.groupby('zp').mean()
        
        #dfm['SA'] = dfm['SA'].rolling(25).median()
        #dfm['CT'] = dfm['CT'].rolling(25).median()
        dfm['SA'] = dfm['SA'].rolling(50,win_type='hanning').mean()
        dfm['CT'] = dfm['CT'].rolling(50,win_type='hanning').mean()
        #dfm['SA'] = dfm['sigmatheta'].rolling(25,win_type='hanning').mean()
        
        Nsq,pr_mid = gsw.Nsquared(dfm['SA'],dfm['CT'],dfm.index,lat=mlat)
        N = np.sqrt(Nsq)
        
        if i==0:
            dfall = dfm
        else:
            dfall = dfall.append(dfm)
        
        hrov, = ax1.plot(dfm['sigma4'],dfm.index,color='gray',lw=0.5)
        ax2.semilogx(N,pr_mid,color='gray',lw=0.5)
        
        cnt = cnt+1
        print(np.array(dfm['year'])[0],np.array(dfm['day'])[0])
    except:
        pass

hwod, = ax1.plot(msigma4,mpr,'k-',lw=2)
ax1.invert_yaxis()
ax1.legend((hrov,hwod),('ROV','World Ocean Atlas'))
ax1.set_ylabel('pressure [dbar]')
ax1.set_xlabel('$\sigma_4$ [kg/m$^3$]')
ax1.set_title('potential density\n4000 dbar reference')
xl = ax1.get_xlim()
yl = ax1.get_ylim()
ax1.set_ylim([yl[0],0])
ax1.text(xl[0]+0.04*np.diff(xl),yl[1]-0.1*np.diff(yl),'a)')

ax2.semilogx(mN,mpr_mid,'k-',lw=2)
ax2.invert_yaxis()
ax2.get_yaxis().set_ticklabels([])
ax2.set_title('buoyancy frequency')
ax2.set_xlabel('$N$ [rad s$^{-1}$]')
xl = ax2.get_xlim()
yl = ax2.get_ylim()
ax2.set_ylim([yl[0],0])
ax2.text(xl[0]+0.001*np.diff(xl),yl[1]-0.1*np.diff(yl),'b)')
plt.tight_layout()
plt.savefig('figures_paper/water_column_strat.pdf')
plt.savefig('figures_paper/water_column_strat.png')

FigureCanvasNbAgg()

NameError: name 'file_list' is not defined

In [ ]:
yl

In [ ]:
0.0007*86400

In [ ]:
np.array(dfm['year'])[0]